In [1]:
from PIL import Image
import numpy
import os
import re
from resizeimage import resizeimage

In [2]:
# resize images
# resizeImages('../data/yalefaces/', 'subject01')
def resizeImages(path, prefix):    
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(path + name)
                resizedImg = resizeimage.resize_height(img, 50)
                resizedImg.save('yalefaces_resized/'+name)

# images into array: http://stackoverflow.com/questions/13550376/pil-image-to-array-numpy-array-to-array-python
def readImagesIntoMatrix(path, prefix):
    images = None
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(path + name)
                (width, height) = img.size
                imgArr = numpy.asarray(img).reshape(height * width)
                if images is not None:
                    images = numpy.vstack([images, imgArr])
                else:
                    images = imgArr
    return images

In [3]:
resizeImages('yalefaces/', 'subject.*')
imageList = readImagesIntoMatrix('yalefaces_resized/', 'subject.*happy').T

Image.fromarray(numpy.uint8(imageList.T[0].reshape(50,66))).save("test/testsaveimage1.gif")


In [4]:
(width, height) = 66, 50
# average Image
avgImg =  numpy.mean(imageList, axis=1)
Image.fromarray(numpy.uint8(avgImg.reshape(50,66))).save('test/avgImg.gif')

In [ ]:
# Subtract mean from each column (which are rows of transpose, how numpy does it)
imagesDiff = (imageList.T - avgImg).T

# Get EigenVectors
covMatrix = numpy.cov(imagesDiff, rowvar=True)
eigenVal, eigenVec = numpy.linalg.eigh(covMatrix)

# Sort eigenvectors
# http://stackoverflow.com/questions/8092920/sort-eigenvalues-and-associated-eigenvectors-after-using-numpy-linalg-eig-in-pyt
idx = eigenVal.argsort()[::-1]
eigenVal = eigenVal[idx]
eigenVec = eigenVec[:,idx]

In [ ]:
# Testing the eigenvectors
# Taken from http://sebastianraschka.com/Articles/2014_pca_step_by_step.html
for i in range(len(eigenVal)):
    eigv = eigenVec[:,i].reshape(1,3300).T
    numpy.testing.assert_array_almost_equal(covMatrix.dot(eigv), eigenVal[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)

# Also lets assert the eigenvectors are orthogonal
for i in range(len(eigenVal)):
    for j in range(len(eigenVal)):
        if i < j:
            break # Comment this out if you want to run this silly test
            # Commented out because if you run this it takes FOREVER.
            # numpy.testing.assert_almost_equal(0, eigenVec[i].T.dot(eigenVec[j]))

In [ ]:
def normalize_face(vector):
    minValue = numpy.amin(vector)
    maxValue = numpy.amax(vector)
    scale = (255./(maxValue-minValue))
    return (vector - minValue)*scale

for i in range (1, 100):
    eigv = eigenVec[:,i].reshape(1,3300).T
    eigenface = normalize_face(eigv)
    Image.fromarray(numpy.uint8(eigenface.reshape(50,66))).save('test/happy_eigenface%s.gif' % i)



In [ ]:
def top_k_eigenface(image, eigenVectors, k):
    start_vector = numpy.zeros((50*66))
    for i in range(0, k-1):
        eigv = eigenVec[:,i].reshape(3300)
        weight = numpy.inner(image, eigv)
        start_vector = start_vector + weight * eigv
    return start_vector

for i in range(0, len(imagesDiff.T)):
    projection = top_k_eigenface(imagesDiff.T[i].reshape(3300), eigv, 13)+avgImg
    print ('Projection dimensions %s' % (projection.shape,))
    Image.fromarray(numpy.uint8(projection.reshape(50,66))).save('test/happy_13_projection_of_subject%s.gif' % i)
    projection = top_k_eigenface(imagesDiff.T[i].reshape(3300), eigv, 3300)+avgImg
    Image.fromarray(numpy.uint8(projection.reshape(50,66))).save('test/happy_3300_projection_of_subject%s.gif' % i)